In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model

# reflect the tables

In [ ]:
# View all of the classes that automap found


In [ ]:
# Save references to each table

In [ ]:
# Create our session (link) from Python to the DB

## Bonus Challenge Assignment: Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set

In [ ]:
# Plot the results from your previous query as a bar chart. 
fig, ax = plt.subplots(figsize=(7, 5))

# Use the average temperature for bar height (y value)
y_value = trip_results[0][1]

# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
yerr = trip_tmax - trip_tmin

# Plot the result
x_axis = np.arange(len(trip_results[0]))
plt.xticks([])
plt.bar(x_axis[0], y_value, yerr=yerr, alpha=0.75, color='coral', width=0.5)
plt.ylabel('Temperature (F)', fontweight='bold', fontsize=13)
plt.xlim(-1, 1)
plt.ylim(0,110)

# Use "Trip Avg Temp" as your Title
plt.title('Trip Avg Temp', fontweight='bold', fontsize=14)


### Daily Rainfall Average

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
trip_prcp = session.query(Measurement.station, Station.name,
                          Station.latitude, Station.longitude, 
                          Station.elevation, func.sum(Measurement.prcp))\
                            .filter(Measurement.date <= trip_end_date)\
                            .filter(Measurement.date >= trip_strt_date)\
                            .group_by(Measurement.station)\
                            .order_by(func.sum(Measurement.prcp).desc())\
                            .filter(Measurement.station == Station.station).all()

# Store the result into data frame
trip_prcp_df = pd.DataFrame(trip_prcp, columns=['Station ID', 
                                                'Name', 
                                                'Latitude', 
                                                'Longitude', 
                                                'Elevation', 
                                                'Total Rainfall'])

# Display data
trip_prcp_df


### Daily Temperature Normals

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates
trip_dates = pd.date_range(start_date, end_date)

# Strip off the year and save a list of strings in the format %m-%d
trip_forcast_dates = [trip_dates[d].strftime("%m-%d")  for d in range(len(trip_dates))]


# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.
normals = []
for dm in range(len(trip_forcast_dates)):
    trip_result = daily_normals(trip_forcast_dates[dm])
    normals.append(trip_result[0])
    
# Temperature list for dataframe
trip_tmin = [normals[tm][0] for tm in range(len(normals))]
trip_tmax = [normals[tM][2] for tM in range(len(normals))]
trip_tavg = [normals[ta][1] for ta in range(len(normals))]

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
trip_temp_df = pd.DataFrame({
    'Date': trip_dates,
    'TMin': trip_tmin,
    'TAvg': trip_tavg,
    'TMax': trip_tmax
    
}).set_index('Date')

trip_temp_df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
trip_temp_plot = trip_temp_df.plot(kind='area', stacked=False, rot=45, use_index=False, alpha=0.3, figsize=(7,5))
positions = list(np.arange(len(trip_dates)))
labels = trip_dates.strftime("%Y-%m-%d")
plt.xticks(positions, labels)
plt.xlabel('Date', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (F)', fontweight='bold', fontsize=13)
plt.title('Trip: Temperature Forecast', fontweight='bold', fontsize=14)
plt.legend(loc='best', bbox_to_anchor=(-0.2, -0.1, 0.5, 0.5))

# Save the figure and show
plt.tight_layout()
plt.savefig('Images/trip_plot.png')
plt.show()

## Close Session